In [32]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar  1 12:06:06 2020

@author: Naveen
"""

import gzip
import os
import numpy as np
from keras.datasets import mnist
from selfTraining import selfTrainModel, fullModel
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

#os.chdir(r'C:\Users\Naveen\Google Drive\Heinz\Coursework\Semester 2\Machine Learning\Project')
#z = gzip.open('./MNIST Data/train-images-idx3-ubyte.gz', 'r')
#image_size = 28
#num_images = 60000
#buf = z.read(image_size * image_size * num_images)
#data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
#xTrain = data.reshape(num_images, image_size, image_size)
#xTrain = xTrain.reshape(60000, 784)
#xTrain = xTrain/255.0
#
#z = gzip.open('./MNIST Data/train-labels-idx1-ubyte.gz','r')
#buf = z.read(num_images)
#yTrain = np.frombuffer(buf, dtype = np.uint8).astype(np.int32)
#del(data)
#del(z)
#del(buf)
(xTrain, yTrain), (xTest, yTest) = mnist.load_data()
xTrain = xTrain.reshape(60000, 784)
xTrain = xTrain/255.0
xTest = xTest.reshape(10000, 784)
xTest = xTest/255.0

#xTrain = xTrain[0:5000, :]
#yTrain = yTrain[0:5000]
print(np.unique(yTrain, return_counts=True))

batchSize = 32
maxIter = 5
pThresh = 0.95
labeledSize = 100
selectionType = "Group"

labelSize = [100, 500, 1000, 2000, 5000, 10000, 25000, 50000]
selfTrainErr = []
for size in labelSize:
    output = selfTrainModel(xTrain, yTrain, batchSize, maxIter, pThresh, size, selectionType)
    wrongLabels = output[1]
    yHat = output[0].predict_classes(xTest, batch_size = batchSize)
    cm = confusion_matrix(yTest, yHat)
    selfTrainErr = np.append(selfTrainErr, 1 - sum(np.diag(cm))/sum(sum(cm)))
    print(wrongLabels)
    
model = fullModel(xTrain, yTrain, batchSize)
yHat = model.predict_classes(xTest, batch_size = batchSize)
cm = confusion_matrix(yTest, yHat)
fullErr = 1 - sum(np.diag(cm))/sum(sum(cm))
print(fullErr)
fullErr = np.repeat(fullErr, len(labelSize))
plt.plot(labelSize, selfTrainErr, color = 'red', label = "Self Training")
plt.plot(labelSize, fullErr, color = "blue", label = "Supervised Learning")
plt.xlabel("Number of Labeled Samples")
plt.ylabel("Test Error")
plt.legend()


(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8), array([5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949],
      dtype=int64))
0
59990
Train on 9 samples, validate on 1 samples
Epoch 1/5
 - 1s - loss: 2.3003 - accuracy: 0.0000e+00 - val_loss: 3.6974 - val_accuracy: 0.0000e+00
Epoch 2/5
 - 0s - loss: 0.9681 - accuracy: 1.0000 - val_loss: 5.1479 - val_accuracy: 0.0000e+00
Epoch 3/5
 - 0s - loss: 0.3555 - accuracy: 1.0000 - val_loss: 5.7705 - val_accuracy: 0.0000e+00
Epoch 4/5
 - 0s - loss: 0.2623 - accuracy: 1.0000 - val_loss: 5.7128 - val_accuracy: 0.0000e+00
Epoch 5/5
 - 0s - loss: 0.2427 - accuracy: 0.8889 - val_loss: 5.5196 - val_accuracy: 0.0000e+00
1
58849
Train on 1035 samples, validate on 116 samples
Epoch 1/5
 - 1s - loss: 0.5316 - accuracy: 0.8763 - val_loss: 0.2258 - val_accuracy: 0.9397
Epoch 2/5
 - 0s - loss: 0.1976 - accuracy: 0.9430 - val_loss: 0.0792 - val_accuracy: 0.9828
Epoch 3/5
 - 0s - loss: 0.1239 - accuracy: 0.9604 - val_loss: 0.2279 - val_accuracy: 0

KeyboardInterrupt: 